In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import gc
import sys
from PIL import Image
import cv2
import math, random
import numpy as np
import pandas as pd
import glob
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from collections import OrderedDict
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import load_model
import albumentations as A
from sklearn.model_selection import KFold
import re
import pydicom
from typing import Optional
import torch
from torch.utils.data import Dataset, DataLoader  # Import Dataset here
import torch.nn as nn
import torch.optim as optim
import glob
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import timm


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.17 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
US1_J2KR.dcm:   0%|          | 38.0/154k [00:00<01:15, 2.03kB/s]
MR-SIEMENS-DICOM-WithOverlays.dcm:   0%|          | 125/511k [00:00<01:48, 4.72kB/s]
OBXXXX1A.dcm:   0%|          | 119/486k [00:00<01:39, 4.89kB/s]
US1_UNCR.dcm:   0%|          | 226/923k [00:00<02:35, 5.93kB/s]
color3d_jpeg_baseline.dcm:   0%|          | 1.50k/6.14M [00:00<07:04, 14.5kB/s]


In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

In [3]:
#Below is working se
import pandas as pd
import os
import pydicom
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the necessary CSV files
df_train = pd.read_csv("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv")
df_train_label = pd.read_csv("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv")
df_train_series = pd.read_csv("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv")
df_train = df_train.fillna(0.0)
# Merge the DataFrames for labels and series descriptions
df_labels = pd.merge(df_train_label, df_train_series, on=['study_id', 'series_id'])

# Define the custom order for series_description
order = ['Sagittal T2/STIR', 'Sagittal T1', 'Axial T2']

# Convert the series_description column to a categorical type with the custom order
df_labels['series_description'] = pd.Categorical(df_labels['series_description'], 
                                                 categories=order, 
                                                 ordered=True)

# Sort the DataFrame by study_id and series_description (following the custom order)
df_labels_sorted = df_labels.sort_values(by=['study_id', 'series_description']).reset_index(drop=True)

# Filter out rows with missing or invalid bounding box coordinates AFTER sorting
df_labels_filtered = df_labels_sorted.dropna(subset=['x', 'y'])
df_labels_filtered = df_labels_filtered[(df_labels_filtered['x'] != 0) & (df_labels_filtered['y'] != 0)]

# Path to the folder containing DICOM images
train_images = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/'



In [4]:
#this arranges all the series description in order, and displays the relevant images which have bounding boxes with x,y
df_labels_filtered

,study_id,series_id,instance_number,condition,level,x,y,series_description
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Sagittal T2/STIR
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Sagittal T2/STIR
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Sagittal T2/STIR
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,Sagittal T2/STIR
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,Sagittal T2/STIR
...,...,...,...,...,...,...,...,...
48687,4290709089,3390218084,10,Right Subarticular Stenosis,L3/L4,302.875911,356.304937,Axial T2
48688,4290709089,3390218084,15,Left Subarticular Stenosis,L4/L5,348.203719,341.369400,Axial T2
48689,4290709089,3390218084,15,Right Subarticular Stenosis,L4/L5,305.745866,340.520184,Axial T2
48690,4290709089,3390218084,20,Left Subarticular Stenosis,L5/S1,355.777684,368.960270,Axial T2


In [5]:
# Select the first 30 unique study_ids
first_20_study_ids = df_labels_filtered['study_id'].unique()[:20]

# Filter df_labels_filtered to keep only rows with the first 30 study_ids while retaining all columns
df_series_20 = df_labels_filtered[df_labels_filtered['study_id'].isin(first_20_study_ids)]

# Save the filtered DataFrame with all columns to a new CSV file
df_series_20.to_csv('df_first_20_study_ids.csv', index=False)

In [6]:
df_series_20

,study_id,series_id,instance_number,condition,level,x,y,series_description
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Sagittal T2/STIR
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Sagittal T2/STIR
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Sagittal T2/STIR
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,Sagittal T2/STIR
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,Sagittal T2/STIR
...,...,...,...,...,...,...,...,...
491,53418228,1921001089,17,Right Subarticular Stenosis,L3/L4,131.597663,139.623736,Axial T2
492,53418228,1921001089,23,Left Subarticular Stenosis,L2/L3,161.723884,144.257143,Axial T2
493,53418228,1921001089,23,Right Subarticular Stenosis,L2/L3,137.661660,147.304799,Axial T2
494,53418228,1921001089,29,Left Subarticular Stenosis,L1/L2,164.638170,151.057143,Axial T2


In [7]:
#get images with that only contain instance number in df_labels_fltered

In [8]:
#get train_csv that only has study_id in df_labels_fltered

In [9]:
import pandas as pd

# Filter df_train by matching study_ids in df_series_30
df_train_20 = df_train[df_train['study_id'].isin(df_series_20['study_id'].unique())]

# Display the filtered DataFrame
df_train_20


,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1,4646740,Normal/Mild,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate,Normal/Mild
2,7143189,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
3,8785691,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4,10728036,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild
5,11340341,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild
6,11943292,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
7,13317052,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate
8,22191399,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Moderate
9,26342422,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild


In [10]:
import pandas as pd

# Map categories to integers
label_map = {'Normal/Mild': 0.0, 'Moderate': 1.0, 'Severe': 2.0}

# Convert targets in train.csv to numeric form
def prepare_labels(train_csv):
    # Replace the text labels with numbers for all target columns
    target_columns = train_csv.columns[1:]  # Exclude 'study_id' column
    for col in target_columns:
        train_csv[col] = train_csv[col].map(label_map)
    
    # Fill any remaining NaN values with 0.0
    train_csv.fillna(0.0, inplace=True)
    
    return train_csv

In [11]:
train_csv = prepare_labels(df_train_20)

train_csv

/tmp/ipykernel_17/887935434.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_csv[col] = train_csv[col].map(label_map)
/tmp/ipykernel_17/887935434.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_csv[col] = train_csv[col].map(label_map)
/tmp/ipykernel_17/887935434.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4646740,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0
2,7143189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8785691,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10728036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,11340341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
6,11943292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,13317052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
8,22191399,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0
9,26342422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# img_width, img_height = 128, 128
# depth = 50  # Number of slices per 3D image

NEED TO RESIZE IMAGES TO STANDARD SIZE YET RESIZE BOUDING BOXES CORRECTLY TO 

In [13]:
# image_dir = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'

# def load_images(df_filtered, image_dir, depth=50, target_size=(128,128)):
#     images = []
#     max_height = 0
#     max_width = 0
    
#     # Loop through each study_id in df_filtered
#     for study_id in df_filtered['study_id'].unique():
#         study_filtered = df_filtered[df_filtered['study_id'] == study_id]
        
#         print(f'study id: {study_id}')

#         # Path to the study directory
#         study_id_path = os.path.join(image_dir, str(study_id))
        
#         # Ensure the study path exists
#         if not os.path.isdir(study_id_path):
#             continue

#         # Get the ordered series IDs (this should be based on a specific ordering logic)
#         series_ids_ordered = study_filtered['series_id'].unique()

#         # Loop through each ordered series_id
#         for series_id in series_ids_ordered:
#             series_id_path = os.path.join(study_id_path, str(series_id))
            
#             # Check if the series_id folder exists
#             if not os.path.isdir(series_id_path):
#                 continue

#             # List and sort all DICOM files in the current series_id folder
#             dicom_files = [f for f in os.listdir(series_id_path) if f.endswith('.dcm')]
#             dicom_files = sorted(dicom_files, key=lambda x: int(x.replace('.dcm', '')))

#             # Filter labels for the current series_id
#             df_series_filtered = study_filtered[study_filtered['series_id'] == series_id]

#             # Skip if no relevant labels exist for this series_id
#             if df_series_filtered.empty:
#                 continue
            
#             print(f'series id: {series_id}')
            
#             # Get the series description (assume it's the same for all files in the series_id)
#             series_description = df_series_filtered['series_description'].iloc[0]
            
#             print(f'{series_description}')
            
#             stacked_images = []
#             processed_instance_numbers = set()  # Track unique instance_numbers

#             # Loop through each DICOM file in the series_id folder
#             for dicom_file in dicom_files:
#                 dicom_path = os.path.join(series_id_path, dicom_file)

#                 # Read the DICOM file
#                 dicom_data = pydicom.dcmread(dicom_path)

#                 # Access the pixel data
#                 image_data = dicom_data.pixel_array

#                 # Convert the pixel data to 8-bit image for OpenCV
#                 image_8bit = cv2.normalize(image_data, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

#                 # Convert grayscale image to 3-channel BGR format for colored annotations
#                 image_bgr = cv2.cvtColor(image_8bit, cv2.COLOR_GRAY2BGR)

#                 # Get the corresponding instance number (matching the DICOM file)
#                 instance_number = int(dicom_file.replace('.dcm', ''))

#                 # Skip if the instance_number has already been processed
#                 if instance_number in processed_instance_numbers:
#                     continue

#                 # Get the rows in df_series_filtered that match this instance_number
#                 relevant_labels = df_series_filtered[df_series_filtered['instance_number'] == instance_number]

#                 # If there are no relevant labels for this instance, skip the image
#                 if relevant_labels.empty:
#                     continue
                    
#                 # Get original dimensions
#                 original_height, original_width = image_bgr.shape[:2]

#                 # Resize the image to the target size (128x128)
#                 resized_image_bgr = cv2.resize(image_bgr, target_size, interpolation=cv2.INTER_LINEAR)

#                 # Calculate the scaling factors for width and height
#                 scale_x = target_size[1] / original_width
#                 scale_y = target_size[0] / original_height

#                 # Draw circles at the x, y coordinates in the labels
#                 for _, row in relevant_labels.iterrows():
#                     x, y = int(row['x']), int(row['y'])
#                     # Draw the circle on the image (with a radius of 10, color red, and thickness 2)
#                     cv2.circle(image_bgr, (x, y), radius=10, color=(255, 255, 255), thickness=2)
#                     # Display the image with circles using matplotlib (optional for visual inspection)
                    
#                     # Scale the coordinates based on the resizing
#                     scaled_x = int(x * scale_x)
#                     scaled_y = int(y * scale_y)

#                     # Draw the circle on the resized image (with a radius of 10, color red, and thickness 2)
#                     cv2.circle(resized_image_bgr, (scaled_x, scaled_y), radius=10, color=(0, 0, 255), thickness=2)
                    
#                 plt.figure()
#                 plt.imshow(cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB))
#                 plt.title(f'{dicom_file} - {series_description}')
#                 plt.show()

#                 # Track maximum width and height of images for later padding
#                 max_height = max(max_height, image_bgr.shape[0])
#                 max_width = max(max_width, image_bgr.shape[1])

#                 # Append the processed image to the stacked_images list (only once per unique instance_number)
#                 stacked_images.append(image_bgr)
#                 processed_instance_numbers.add(instance_number)
                
#             # Stack the images to create a 3D array for this series_id
#             if stacked_images:
#                 stacked_images_np = np.stack(stacked_images, axis=-1)

#                 # If the number of slices is less than depth, pad with zeros
#                 if stacked_images_np.shape[-1] < depth:
#                     pad_width = depth - stacked_images_np.shape[-1]
#                     stacked_images_np = np.pad(stacked_images_np, 
#                                                ((0, 0), (0, 0), (0, 0), (0, pad_width)), 
#                                                mode='constant')
#                 # If more slices, trim the stack
#                 else:
#                     stacked_images_np = stacked_images_np[:, :, :, :depth]

#                 images.append(stacked_images_np)

#     # Final padding to ensure all images have the same height and width
#     padded_images = []
#     for img in images:
#         height_pad = max_height - img.shape[0]
#         width_pad = max_width - img.shape[1]

#         padded_img = np.pad(img, 
#                             ((0, height_pad), (0, width_pad), (0, 0), (0, 0)),  # Pad height and width dimensions
#                             mode='constant')
#         padded_images.append(padded_img)

#     return np.array(padded_images)

In [14]:
# #TO CHECK ALL IMAGES HAVE POSITION AND BOUNDING BOXES 

# image_dir = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'

# def load_images(df_filtered, image_dir, target_size=(128, 128), depth=50):
#     images = []
#     max_height = 0
#     max_width = 0
    
#     # Loop through each study_id in df_filtered
#     for study_id in df_filtered['study_id'].unique():
#         study_filtered = df_filtered[df_filtered['study_id'] == study_id]
        
#         print(f'study id: {study_id}')

#         # Path to the study directory
#         study_id_path = os.path.join(image_dir, str(study_id))
        
#         # Ensure the study path exists
#         if not os.path.isdir(study_id_path):
#             continue

#         # Get the ordered series IDs (this should be based on a specific ordering logic)
#         series_ids_ordered = study_filtered['series_id'].unique()

#         # Loop through each ordered series_id
#         for series_id in series_ids_ordered:
#             series_id_path = os.path.join(study_id_path, str(series_id))
            
#             # Check if the series_id folder exists
#             if not os.path.isdir(series_id_path):
#                 continue

#             # List and sort all DICOM files in the current series_id folder
#             dicom_files = [f for f in os.listdir(series_id_path) if f.endswith('.dcm')]
#             dicom_files = sorted(dicom_files, key=lambda x: int(x.replace('.dcm', '')))

#             # Filter labels for the current series_id
#             df_series_filtered = study_filtered[study_filtered['series_id'] == series_id]

#             # Skip if no relevant labels exist for this series_id
#             if df_series_filtered.empty:
#                 continue
            
#             print(f'series id: {series_id}')
            
#             # Get the series description (assume it's the same for all files in the series_id)
#             series_description = df_series_filtered['series_description'].iloc[0]
            
#             print(f'{series_description}')
            
#             stacked_images = []
#             processed_instance_numbers = set()  # Track unique instance_numbers

#             # Loop through each DICOM file in the series_id folder
#             for dicom_file in dicom_files:
#                 dicom_path = os.path.join(series_id_path, dicom_file)

#                 # Read the DICOM file
#                 dicom_data = pydicom.dcmread(dicom_path)

#                 # Access the pixel data
#                 image_data = dicom_data.pixel_array

#                 # Convert the pixel data to 8-bit image for OpenCV
#                 image_8bit = cv2.normalize(image_data, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

#                 # Convert grayscale image to 3-channel BGR format for colored annotations
#                 image_bgr = cv2.cvtColor(image_8bit, cv2.COLOR_GRAY2BGR)

#                 # Get the corresponding instance number (matching the DICOM file)
#                 instance_number = int(dicom_file.replace('.dcm', ''))

#                 # Skip if the instance_number has already been processed
#                 if instance_number in processed_instance_numbers:
#                     continue

#                 # Get the rows in df_series_filtered that match this instance_number
#                 relevant_labels = df_series_filtered[df_series_filtered['instance_number'] == instance_number]

#                 # If there are no relevant labels for this instance, skip the image
#                 if relevant_labels.empty:
#                     continue

#                 # Draw circles at the x, y coordinates in the labels
#                 for _, row in relevant_labels.iterrows():
#                     x, y = int(row['x']), int(row['y'])
#                     # Draw the circle on the image (with a radius of 10, color red, and thickness 2)
#                     cv2.circle(image_bgr, (x, y), radius=10, color=(0, 0, 255), thickness=2)
#                     # Display the image with circles using matplotlib (optional for visual inspection)
#                 plt.figure()
#                 plt.imshow(cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB))
#                 plt.title(f'{dicom_file} - {series_description}')
#                 plt.show()

#                 # Track maximum width and height of images for later padding
#                 max_height = max(max_height, image_bgr.shape[0])
#                 max_width = max(max_width, image_bgr.shape[1])

#                 # Append the processed image to the stacked_images list (only once per unique instance_number)
#                 stacked_images.append(image_bgr)
#                 processed_instance_numbers.add(instance_number)
                
#             # Stack the images to create a 3D array for this series_id
#             if stacked_images:
#                 stacked_images_np = np.stack(stacked_images, axis=-1)

#                 # If the number of slices is less than depth, pad with zeros
#                 if stacked_images_np.shape[-1] < depth:
#                     pad_width = depth - stacked_images_np.shape[-1]
#                     stacked_images_np = np.pad(stacked_images_np, 
#                                                ((0, 0), (0, 0), (0, 0), (0, pad_width)), 
#                                                mode='constant')
#                 # If more slices, trim the stack
#                 else:
#                     stacked_images_np = stacked_images_np[:, :, :, :depth]

#                 images.append(stacked_images_np)

#     # Final padding to ensure all images have the same height and width
#     padded_images = []
#     for img in images:
#         height_pad = max_height - img.shape[0]
#         width_pad = max_width - img.shape[1]

#         padded_img = np.pad(img, 
#                             ((0, height_pad), (0, width_pad), (0, 0), (0, 0)),  # Pad height and width dimensions
#                             mode='constant')
#         padded_images.append(padded_img)

#     return np.array(padded_images)


In [15]:
# # TO CHECK ALL IMAGES HAVE POSITION AND BOUDING BOXES REDRAWN, BUT HAVING DIFF SHAPES

# def load_images(df_filtered, image_dir, depth=50, target_size=(128,128)):
#     images = []
#     max_height = 0
#     max_width = 0
    
#     for study_id in df_filtered['study_id'].unique():
#         study_filtered = df_filtered[df_filtered['study_id'] == study_id]
#         print(f'study id: {study_id}')
#         study_id_path = os.path.join(image_dir, str(study_id))
        
#         if not os.path.isdir(study_id_path):
#             continue

#         series_ids_ordered = study_filtered['series_id'].unique()
#         for series_id in series_ids_ordered:
#             series_id_path = os.path.join(study_id_path, str(series_id))
            
#             if not os.path.isdir(series_id_path):
#                 continue

#             dicom_files = [f for f in os.listdir(series_id_path) if f.endswith('.dcm')]
#             dicom_files = sorted(dicom_files, key=lambda x: int(x.replace('.dcm', '')))

#             df_series_filtered = study_filtered[study_filtered['series_id'] == series_id]

#             if df_series_filtered.empty:
#                 continue
            
#             print(f'series id: {series_id}')
#             series_description = df_series_filtered['series_description'].iloc[0]
#             print(f'{series_description}')
            
#             stacked_images = []
#             processed_instance_numbers = set()

#             for dicom_file in dicom_files:
#                 dicom_path = os.path.join(series_id_path, dicom_file)
#                 dicom_data = pydicom.dcmread(dicom_path)
#                 image_data = dicom_data.pixel_array
#                 image_8bit = cv2.normalize(image_data, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                
#                 # Convert to grayscale (1 channel)
#                 image_gray = image_8bit
                
#                 # Resize to the target size (128, 128)
#                 resized_image_gray = cv2.resize(image_gray, target_size, interpolation=cv2.INTER_LINEAR)

#                 instance_number = int(dicom_file.replace('.dcm', ''))

#                 if instance_number in processed_instance_numbers:
#                     continue

#                 relevant_labels = df_series_filtered[df_series_filtered['instance_number'] == instance_number]

#                 if relevant_labels.empty:
#                     continue

#                 for _, row in relevant_labels.iterrows():
#                     x, y = int(row['x']), int(row['y'])
#                     scaled_x = int(x * (target_size[1] / image_gray.shape[1]))
#                     scaled_y = int(y * (target_size[0] / image_gray.shape[0]))
#                     cv2.circle(resized_image_gray, (scaled_x, scaled_y), radius=10, color=255, thickness=2)

#                 plt.figure()
#                 plt.imshow(resized_image_gray, cmap='gray')
#                 plt.title(f'{dicom_file} - {series_description}')
#                 plt.show()
                
#                 stacked_images.append(resized_image_gray)
#                 processed_instance_numbers.add(instance_number)
            
#             if stacked_images:
#                 stacked_images_np = np.stack(stacked_images, axis=-1)

#                 if stacked_images_np.shape[-1] < depth:
#                     pad_width = depth - stacked_images_np.shape[-1]
#                     stacked_images_np = np.pad(stacked_images_np, ((0, 0), (0, 0), (0, pad_width)), mode='constant')
#                 else:
#                     stacked_images_np = stacked_images_np[:, :, :depth]

#                 images.append(stacked_images_np)

#     padded_images = []
#     for img in images:
#         height_pad = target_size[0] - img.shape[0]
#         width_pad = target_size[1] - img.shape[1]

#         padded_img = np.pad(img, ((0, height_pad), (0, width_pad), (0, 0)), mode='constant')
        
#         # Add a new axis for the single channel (grayscale)
#         padded_img = np.expand_dims(padded_img, axis=-2)
        
#         padded_images.append(padded_img)

#     return np.array(padded_images)

In [16]:
#IMAGES WITH POSITION AND BOUNDING BOXES STACKED AT FIXED SHAPE

import os
import numpy as np
import cv2
import pydicom
import matplotlib.pyplot as plt

def load_images(df_filtered, image_dir, depth=50, target_size=(128, 128)):
    images = []
    max_height = 0
    max_width = 0
    
    for study_id in df_filtered['study_id'].unique():
        study_filtered = df_filtered[df_filtered['study_id'] == study_id]
        print(f'Study ID: {study_id}')
        study_id_path = os.path.join(image_dir, str(study_id))
        
        if not os.path.isdir(study_id_path):
            continue
        
        stacked_images = []
        processed_instance_numbers = set()

        series_ids_ordered = study_filtered['series_id'].unique()
        for series_id in series_ids_ordered:
            series_id_path = os.path.join(study_id_path, str(series_id))
            
            if not os.path.isdir(series_id_path):
                continue

            dicom_files = [f for f in os.listdir(series_id_path) if f.endswith('.dcm')]
            dicom_files = sorted(dicom_files, key=lambda x: int(x.replace('.dcm', '')))
            
            df_series_filtered = study_filtered[study_filtered['series_id'] == series_id]

            if df_series_filtered.empty:
                continue
            
            print(f'Series ID: {series_id}')
            series_description = df_series_filtered['series_description'].iloc[0]
            print(f'Description: {series_description}')
            
            for dicom_file in dicom_files:
                dicom_path = os.path.join(series_id_path, dicom_file)
                dicom_data = pydicom.dcmread(dicom_path)
                image_data = dicom_data.pixel_array
                image_8bit = cv2.normalize(image_data, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                
                # Convert to grayscale (1 channel)
                image_gray = image_8bit
                
                # Resize to the target size (128, 128)
                resized_image_gray = cv2.resize(image_gray, target_size, interpolation=cv2.INTER_LINEAR)

                instance_number = int(dicom_file.replace('.dcm', ''))

                if instance_number in processed_instance_numbers:
                    continue

                relevant_labels = df_series_filtered[df_series_filtered['instance_number'] == instance_number]

                if relevant_labels.empty:
                    continue

                for _, row in relevant_labels.iterrows():
                    x, y = int(row['x']), int(row['y'])
                    scaled_x = int(x * (target_size[1] / image_gray.shape[1]))
                    scaled_y = int(y * (target_size[0] / image_gray.shape[0]))
                    cv2.circle(resized_image_gray, (scaled_x, scaled_y), radius=5, color=255, thickness=2)
                
#                 plt.figure()
#                 plt.imshow(resized_image_gray, cmap='gray')
#                 plt.title(f'{dicom_file} - {series_description}')
#                 plt.show()
                
                # Append the resized image to the list for stacking
                stacked_images.append(resized_image_gray)
                processed_instance_numbers.add(instance_number)

        # After processing all series for this study_id, stack the images
        if stacked_images:
            stacked_images_np = np.stack(stacked_images, axis=-1)

            if stacked_images_np.shape[-1] < depth:
                pad_width = depth - stacked_images_np.shape[-1]
                stacked_images_np = np.pad(stacked_images_np, ((0, 0), (0, 0), (0, pad_width)), mode='constant')
            else:
                stacked_images_np = stacked_images_np[:, :, :depth]

            # Add a new axis for the single channel (grayscale)
            stacked_images_np = np.expand_dims(stacked_images_np, axis=-2)
            images.append(stacked_images_np)

    return np.array(images)

# Assuming df_filtered is already loaded with 'study_id', 'series_id', and 'series_description'
X = load_images(df_series_20, image_dir="/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images")
# y = train_csv  # Assuming this contains the corresponding labels for the study_ids

# Check if images and labels are loaded correctly
print(f"Number of images loaded: {len(X)}")
# print(f"Number of labels loaded: {len(y)}")  # Uncomment when y is available

print(X.shape)
# print(y.shape)  # Uncomment when y is available


Study ID: 4003253
Series ID: 702807833
Description: Sagittal T2/STIR
Series ID: 1054713880
Description: Sagittal T1
Series ID: 2448190387
Description: Axial T2
Study ID: 4646740
Series ID: 3666319702
Description: Sagittal T2/STIR
Series ID: 3486248476
Description: Sagittal T1
Series ID: 3201256954
Description: Axial T2
Study ID: 7143189
Series ID: 132939515
Description: Sagittal T2/STIR
Series ID: 3219733239
Description: Sagittal T1
Series ID: 1951927562
Description: Axial T2
Study ID: 8785691
Series ID: 481125819
Description: Sagittal T2/STIR
Series ID: 1570286759
Description: Sagittal T1
Series ID: 2406919186
Description: Axial T2
Study ID: 10728036
Series ID: 3491739931
Description: Sagittal T2/STIR
Series ID: 2399638375
Description: Sagittal T1
Series ID: 142859125
Description: Axial T2
Series ID: 2073726394
Description: Axial T2
Study ID: 11340341
Series ID: 3543553307
Description: Sagittal T2/STIR
Series ID: 2231042680
Description: Sagittal T1
Series ID: 1224932122
Description: A

In [17]:
# # Assuming df_filtered is already loaded with 'study_id', 'series_id', and 'series_description'
# X = load_images(df_series_30, image_dir="/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images", 
#                    img_width=img_width, img_height=img_height, depth=50)

# # X = np.squeeze(X, axis=-2)  # Removes the 4th dimension

# y = train_csv

# # Check if images and labels are loaded correctly
# print(f"Number of images loaded: {len(X)}")
# print(f"Number of labels loaded: {len(y)}")

# print(X.shape)
# print(y.shape)

In [18]:
X = np.squeeze(X, axis=3)
y = train_csv.iloc[:, 1:].values  # Targets from train_csv (excluding 'study_id'

print(X.shape)
print(y.shape)

(20, 128, 128, 50)
(20, 25)


In [19]:

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now X_train and X_test should have the correct shape for your CNN

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout

def model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv3D(32, (3, 3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Conv3D(64, (3, 3, 3), activation='relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Conv3D(128, (3, 3, 3), activation='relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    
    return model

In [21]:
img_width, img_height = 128, 128
depth = 50  # Number of slices per 3D image

In [22]:
# Create and train the model
input_shape = (img_width, img_height, depth, 1)  # Adjusted for 3D input
num_classes = 25
model = model(input_shape, num_classes)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=2, validation_data=(X_test, y_test))


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 58s 7s/step - accuracy: 0.0311 - loss: 16483.3633 - val_accuracy: 0.0000e+00 - val_loss: 153810.5000
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 55s 7s/step - accuracy: 0.0000e+00 - loss: 527342.3750 - val_accuracy: 0.0000e+00 - val_loss: 1773242.0000
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 80s 7s/step - accuracy: 0.0000e+00 - loss: 6682850.0000 - val_accuracy: 0.0000e+00 - val_loss: 26229144.0000
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 82s 7s/step - accuracy: 0.0450 - loss: 53008636.0000 - val_accuracy: 0.0000e+00 - val_loss: 162462176.0000
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 55s 7s/step - accuracy: 0.0561 - loss: 231726784.0000 - val_accuracy: 0.0000e+00 - val_loss: 562139520.0000
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 55s 7s/step - accuracy: 0.0733 - loss: 1148249216.0000 - val_accuracy: 0.0000e+00 - val_loss: 2315204096.0000
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 82s 7s/step - accuracy: 0.0000e+00 - loss: 4741925376.0000 - val_accuracy: 0.0000e+00 - val_loss: 76902471